In [48]:
import pandas as pd
import plotly.express as px

In [ ]:

#! git clone git@github.com:huggingface/pytorch-image-models.git
#! copy the `results` folder so it's accessible here

In [49]:
df_results = pd.read_csv('./results/results-imagenet.csv').sort_values(by='model')
df_results['model_fqn'] = df_results['model']
df_results['model'] = df_results['model'].str.split('.').str[0]
df_results.tail()

,model,top1,top1_err,top5,top5_err,param_count,img_size,crop_pct,interpolation,model_fqn
762,xcit_tiny_24_p16_224,79.448,20.552,94.878,5.122,12.12,224,1.0,bicubic,xcit_tiny_24_p16_224.fb_in1k
459,xcit_tiny_24_p16_384,82.570,17.430,96.276,3.724,12.12,384,1.0,bicubic,xcit_tiny_24_p16_384.fb_dist_in1k
460,xcit_tiny_24_p8_224,82.566,17.434,96.172,3.828,12.11,224,1.0,bicubic,xcit_tiny_24_p8_224.fb_dist_in1k
538,xcit_tiny_24_p8_224,81.892,18.108,95.970,4.030,12.11,224,1.0,bicubic,xcit_tiny_24_p8_224.fb_in1k
327,xcit_tiny_24_p8_384,83.746,16.254,96.710,3.290,12.11,384,1.0,bicubic,xcit_tiny_24_p8_384.fb_dist_in1k


In [50]:
# merging results and benchmark data example
df_benchmark = pd.read_csv(f'./results/benchmark-infer-amp-nhwc-pt111-cu113-rtx3090.csv').merge(df_results, on='model').sort_values(by='model')
# getting the count of models
df_benchmark.tail()

,model,infer_samples_per_sec,infer_step_time,infer_batch_size,infer_img_size,param_count_x,top1,top1_err,top5,top5_err,param_count_y,img_size,crop_pct,interpolation,model_fqn
403,xcit_tiny_12_p8_224,1260.86,812.129,1024,224,6.71,79.688,20.312,95.054,4.946,6.71,224,1.0,bicubic,xcit_tiny_12_p8_224.fb_in1k
253,xcit_tiny_24_p16_224,2556.28,400.571,1024,224,12.12,79.448,20.552,94.878,5.122,12.12,224,1.0,bicubic,xcit_tiny_24_p16_224.fb_in1k
252,xcit_tiny_24_p16_224,2556.28,400.571,1024,224,12.12,80.454,19.546,95.218,4.782,12.12,224,1.0,bicubic,xcit_tiny_24_p16_224.fb_dist_in1k
521,xcit_tiny_24_p8_224,665.57,1538.510,1024,224,12.11,81.892,18.108,95.970,4.030,12.11,224,1.0,bicubic,xcit_tiny_24_p8_224.fb_in1k
520,xcit_tiny_24_p8_224,665.57,1538.510,1024,224,12.11,82.566,17.434,96.172,3.828,12.11,224,1.0,bicubic,xcit_tiny_24_p8_224.fb_dist_in1k


In [51]:
def merge_models_results_and_benchmark_data(part, col):
    df = pd.read_csv(f'./results/benchmark-{part}-amp-nhwc-pt111-cu113-rtx3090.csv').merge(df_results, on='model')
    # calculate and add a new column 'secs' as the reciprocal of the values in column `col`
    df['secs'] = 1. / df[col]
    df['family'] = df.model.str.extract('^([a-z]+?(?:v2)?)(?:\d|_|$)')
    df.sort_values(by='model', inplace=True)
    return df

In [55]:
# getting the inference results
df_inference_res = merge_models_results_and_benchmark_data('infer', 'infer_samples_per_sec')
df_inference_res.tail()

,model,infer_samples_per_sec,infer_step_time,infer_batch_size,infer_img_size,param_count_x,top1,top1_err,top5,top5_err,param_count_y,img_size,crop_pct,interpolation,model_fqn,secs,family
403,xcit_tiny_12_p8_224,1260.86,812.129,1024,224,6.71,79.688,20.312,95.054,4.946,6.71,224,1.0,bicubic,xcit_tiny_12_p8_224.fb_in1k,0.000793,xcit
253,xcit_tiny_24_p16_224,2556.28,400.571,1024,224,12.12,79.448,20.552,94.878,5.122,12.12,224,1.0,bicubic,xcit_tiny_24_p16_224.fb_in1k,0.000391,xcit
252,xcit_tiny_24_p16_224,2556.28,400.571,1024,224,12.12,80.454,19.546,95.218,4.782,12.12,224,1.0,bicubic,xcit_tiny_24_p16_224.fb_dist_in1k,0.000391,xcit
521,xcit_tiny_24_p8_224,665.57,1538.510,1024,224,12.11,81.892,18.108,95.970,4.030,12.11,224,1.0,bicubic,xcit_tiny_24_p8_224.fb_in1k,0.001502,xcit
520,xcit_tiny_24_p8_224,665.57,1538.510,1024,224,12.11,82.566,17.434,96.172,3.828,12.11,224,1.0,bicubic,xcit_tiny_24_p8_224.fb_dist_in1k,0.001502,xcit


In [58]:
# plotting
w,h = 1000,800

def show_all(df, title, size):
    return px.scatter(df, width=w, height=h, size=df[size]**2, title=title,
        trendline="ols", trendline_options={'log_x':True},
        x='secs',  y='top1', log_x=True, color='family', hover_name='model_fqn', hover_data=[size])

In [63]:
show_all(df_inference_res, 'Inference', 'infer_img_size')

As we can see, the `levit` family models are extremely fast and precise for image recognition.